In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from math import sqrt
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

In [2]:
train_df = pd.read_csv("train.csv", low_memory=False)
test_df = pd.read_csv("test.csv", low_memory=False)
data = pd.read_csv("data_set.csv", low_memory = False)

In [3]:
## los features salen de feature_main , son estas columnas:
##['id', 'days_to_today', 'año', 'antiguedad', 'centroscomercialescercanos', 'escuelascercanas', 'habitaciones', 'garages', 'banos', 'anio', 'Apartamento', 'Bodega comercial', 'Casa', 'Casa en condominio', 'Casa uso de suelo', 'Departamento Compartido', 'Duplex', 'Edificio', 'Garage', 'Hospedaje', 'Huerta', 'Inmuebles productivos urbanos', 'Local Comercial', 'Local en centro comercial', 'Lote', 'Nave industrial', 'Oficina comercial', 'Otros', 'Quinta Vacacional', 'Rancho', 'Terreno', 'Terreno comercial', 'Terreno industrial', 'Villa', 'gimnasio', 'usosmultiples', 'piscina', 'qty_of_extras', 'metroscubiertos', 'metrostotales', 'precio_x_metro_cub_ciudad', 'precio_x_metro_tot_ciudad', 'precio_x_metro_cub_provincia', 'precio_x_metro_tot_provincia', 'precio_x_metro_cub_idzona', 'precio_x_metro_tot_idzona', 'precio_x_metro_cub_tipodepropiedad', 'precio_x_metro_tot_tipodepropiedad', 'popular_desc_1', 'popular_desc_2', 'popular_desc_3', 'popular_desc_4', 'popular_desc_5', 'popular_desc_6', 'popular_desc_7', 'popular_desc_8', 'popular_desc_9', 'popular_desc_10', 'popular_desc_11', 'popular_desc_12', 'popular_desc_13', 'idzona', 'lat', 'lng', 'avenue', 'Aguascalientes', 'Baja California Norte', 'Baja California Sur', 'Campeche', 'Chiapas', 'Chihuahua', 'Coahuila', 'Colima', 'Distrito Federal', 'Durango', 'Edo. de México', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Michoacán', 'Morelos', 'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo', 'San luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas', 'periodo', 'dolar', 'habxmetros', 'habxmetroscub', 'banosxmetros', 'banosxmetroscub', 'banosxhab', 'largotitulo', 'precio_ciudad', 'precio_provincia', 'precio_idzona']

In [4]:
train_df = train_df[['id','precio']]
train_df['precio'].value_counts()

1500000.0    3041
2500000.0    3036
1200000.0    2909
3500000.0    2726
1300000.0    2480
             ... 
2704600.0       1
1352250.0       1
938260.0        1
2704460.0       1
2327000.0       1
Name: precio, Length: 15520, dtype: int64

In [5]:
features = pd.merge(train_df, data, on='id', how='inner')
features = features.fillna(0)
features['precio'].value_counts()

1500000.0    3041
2500000.0    3036
1200000.0    2909
3500000.0    2726
1300000.0    2480
             ... 
2704600.0       1
1352250.0       1
938260.0        1
2704460.0       1
2327000.0       1
Name: precio, Length: 15520, dtype: int64

In [6]:
labels = features['precio']
features = features.drop(['id','precio'], axis=1)
features.fillna(0, inplace=True)

In [7]:
features.replace([np.inf, -np.inf], np.nan, inplace=True)
features.fillna(0, inplace=True)


In [8]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25)

In [9]:
from xgboost import XGBRegressor
regressor = XGBRegressor(objective='reg:squarederror', n_estimators=500, learning_rate=0.04,
                             gamma=1, subsample=0.75, colsample_bytree=0.75, max_depth=10)

regressor.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.75, gamma=1,
             importance_type='gain', learning_rate=0.04, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.75, verbosity=1)

In [10]:
y_preds = regressor.predict(x_test)


In [11]:
print (sqrt(mean_absolute_error(y_test,y_preds)))

679.6381549652748


In [12]:
set_test = pd.merge(data, test_df[['id']],on = "id", how = "inner")
ids = set_test["id"]
set_test = set_test.drop(columns=['id'])

In [13]:
set_test = set_test.fillna(0)

In [14]:
set_test.head()

,days_to_today,año,antiguedad,centroscomercialescercanos,escuelascercanas,habitaciones,garages,banos,anio,Apartamento,...,dolar,habxmetros,habxmetroscub,banosxmetros,banosxmetroscub,banosxhab,largotitulo,precio_ciudad,precio_provincia,precio_idzona
0,2328,2013,29.0,0.0,0.0,3.0,0.0,4.0,2013,0,...,12.7325,0.000000,0.010000,0.000000,0.013333,0.0,49.0,4.496526e+06,3.471705e+06,0.000000e+00
1,1502,2015,0.0,0.0,0.0,1.0,1.0,1.0,2015,1,...,16.5000,0.014925,0.014925,0.014925,0.014925,1.0,36.0,2.028803e+06,2.036020e+06,2.684120e+06
2,1649,2015,0.0,1.0,0.0,2.0,1.0,2.0,2015,1,...,15.3775,0.020000,0.022989,0.020000,0.022989,2.0,78.0,3.407009e+06,3.471705e+06,2.726905e+06
3,1707,2015,2.0,0.0,0.0,2.0,2.0,2.0,2015,1,...,15.3488,0.023256,0.023256,0.023256,0.023256,1.0,33.0,2.585813e+06,2.548742e+06,1.465647e+06
4,2302,2013,10.0,1.0,1.0,2.0,1.0,1.0,2013,0,...,13.3803,0.026316,0.025000,0.013158,0.012500,2.0,56.0,9.147382e+05,2.762465e+06,9.399999e+05


In [15]:
preds_kaggle = regressor.predict(set_test)

In [16]:
df_kaggel = pd.DataFrame(preds_kaggle)
df_kaggel.head()

,0
0,6860413.500
1,1100568.750
2,2239893.000
3,1335281.000
4,750450.875


In [17]:
df_kaggel = pd.DataFrame(df_kaggel)                               
df_kaggel["id"] = ids
df_kaggel.rename(columns = {0: 'target'},inplace = True) 
df_kaggel = df_kaggel[['id','target']]

In [18]:
df_kaggel.head()

,id,target
0,4941,6860413.500
1,51775,1100568.750
2,115253,2239893.000
3,299321,1335281.000
4,173570,750450.875


In [19]:
df_kaggel.to_csv('ResultadosXGBrfe.csv',index=False)

In [20]:
df_kaggel.shape

(60000, 2)

In [21]:
df_kaggel.head()

,id,target
0,4941,6860413.500
1,51775,1100568.750
2,115253,2239893.000
3,299321,1335281.000
4,173570,750450.875
